<a href="https://colab.research.google.com/github/jechavarria12/Parcial_1_Fraude/blob/main/Parcial_1_Fraude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Parcial 1
